<a href="https://colab.research.google.com/github/viandmarket25/Automatic_tagging_of_clothing_in_E-Commerce/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import json


In [ ]:
!pip install transformers

In [7]:

with open('sent-d.json', 'r') as json_file:
    sentiment_data = json.load(json_file)

# Loop over the keys of the dictionary
for key in sentiment_data:
    print(key)

# Define the 6 classes
classes=[]
# Loop over the keys of the dictionary
for key in sentiment_data:
    classes.append(key)
    #print(key)

text=[]

max_d=1500
epochs=50
# Access data associated with each key
for key in sentiment_data:
    data = sentiment_data[key]
    dc=0
    newd=[]
    for d in data:
        if dc<max_d:
            newd.append(d)
            dc+=1

    # use new and general length for all data llist
    text=text+newd
    #print(f"Key: {key}, Data: {data}")


fun
relief
worry
neutral
sadness
anger
love
surprise
fear
happy


In [ ]:

# Create a DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the text
encoded_text = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='tf')

# Convert labels to numerical format
labels = [classes.index(cls) for cls in classes for _ in range(max_d)]

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((encoded_text['input_ids'], labels)).shuffle(len(text)).batch(32)

# Split dataset into train and validation sets
train_size = int(0.8 * len(text))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

# Create a DistilBERT model
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(classes))


# Save the trained model
#model.save_pretrained("distil-senti-bert")
#tokenizer.save_pretrained("distil-senti-bert")

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

# Save the trained model
model.save_pretrained("distil-senti-bert")
tokenizer.save_pretrained("distil-senti-bert")

# Plot accuracy and loss
plt.plot(history.history['accuracy'], label='Training Accuracy')
#  plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
#plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Training and Validation Metrics')
plt.legend()
plt.savefig('metrics_plot.png')
plt.show()

# Enter the while loop for user input and prediction
while True:
    input_text = input("Enter a text for sentiment prediction (or 'exit' to quit): ")
    if input_text == 'exit':
        break
    # Tokenize the input text
    input_encoded = tokenizer.encode_plus(input_text, padding=True, truncation=True, max_length=512, return_tensors='tf')
    # Make prediction
    prediction = model.predict(input_encoded['input_ids'])[0]
    predicted_class = classes[prediction.argmax()]
    print(f"The sentiment of '{input_text}' is '{predicted_class}'")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/50
  7/469 [..............................] - ETA: 2:19:53 - loss: 2.2935 - accuracy: 0.1205

In [ ]:
!zip -r distil-senti-bert.zip distil-senti-bert


In [ ]:
from google.colab import files
files.download('distil-senti-bert.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>